### 3.6 손글씨 숫자 인식

글씨를 추론하는 과정만 진행, 그것이 `Foward Propagation`.

In [20]:
'''
'''
import sys,os
import pickle
sys.path.append(os.pardir) # 부모 디렉토리의 파일을 가져올 수 있도록 설정
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True)

print(x_train.shape)    # (60000, 784)
print(t_train.shape)    # (60000,)
print(x_test.shape)     # (10000, 784)
print(t_test.shape)     # (10000,)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [21]:
'''
MNIST 이미지를 화면으로 불러오기
'''
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28,28)
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


### 3.6.2 신경망의 추론 처리

입력층 뉴런 784개, 출력층 뉴런은 10개로 구성

이유는 28*28 크기의 이미지이므로이고, 0~9까지의 숫자 10개이기 때문에 10개의 출력층 뉴런이 생김

첫번째 은닉층은 50개, 두번째 은닉층의 뉴런은 100개인데 이것은 임의의 값

In [22]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def get_data():
    (x_train, t_train), (x_test, t_test) = \
        load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    
    a1 = np.dot(x,W1)
    z1 = sigmoid(a1)
    a2 = np.dot(z1,W2)
    z2 = sigmoid(a2)
    a3 = np.dot(z2,W3)
    y = a3
    
    return y

In [25]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1
print(f'Accuracy : {float(accuracy_cnt) / len(x)}')

Accuracy : 0.9311


### 3.6.3 배치 처리

In [26]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


In [27]:
x, t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0,len(x),batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
print(f'Accuracy : {float(accuracy_cnt) / len(x)}')

Accuracy : 0.9311


### 4.2.5 왜 손실함수를 설정하는가?

- 손실함수 대신 정확도를 사용하게 되면 불연속적인 값이 나올 수가 있음.

- 손실함수는 미분을 통해 연속적으로 변화하는 접선을 가져올 수 있고,

- 가장 중요한 것은 기울기가 0이 되지 않기 때문임.
